In [1]:
pip install pygame

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 660.6 kB/s eta 0:00:17
   ---------------------------------------- 0.1/10.8 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/10.8 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.3/10.8 MB 1.8 MB/s eta 0:00:06
   - -------------------------------------- 0.5/10.8 MB 2.2 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/10.8 MB 2.7 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/10.8 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/10.8 MB 2.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/10.8 MB 3.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.5/10.8 MB 3.3 MB/s eta 0:00:03
   ------ -----

In [1]:
import time
import threading
import random
from collections import deque
import pygame


pygame.init()


screen_width = 800
screen_height = 600


WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
GREEN = (0, 255, 0)
GRAY = (169, 169, 169)
BLUE = (0, 0, 255)

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Traffic Management Simulation")


font = pygame.font.Font(None, 36)


def get_vehicle_count(signal):
    return random.randint(5, 50)


traffic_lights = {
    'signal_1': 'RED',
    'signal_2': 'RED',
    'signal_3': 'RED',
    'signal_4': 'RED'
}


previous_traffic_lights = traffic_lights.copy()


vehicle_counts = {
    'signal_1': 0,
    'signal_2': 0,
    'signal_3': 0,
    'signal_4': 0
}


recent_green_signals = deque(maxlen=3)


class Vehicle:
    def __init__(self, x, y, lane, direction, signal):
        self.x = x
        self.y = y
        self.lane = lane
        self.direction = direction
        self.signal = signal
        self.speed = random.randint(2, 5)
    
    def can_move(self):
        return traffic_lights[self.signal] == 'GREEN' or (traffic_lights[self.signal] == 'YELLOW' and previous_traffic_lights[self.signal] != 'GREEN')

    def move(self):
        if self.can_move():
            if self.direction == "right":
                self.x += self.speed
                if self.x > screen_width:
                    self.x = -50
            elif self.direction == "left":
                self.x -= self.speed
                if self.x < -50:
                    self.x = screen_width + 50
            elif self.direction == "up":
                self.y -= self.speed
                if self.y < -50:
                    self.y = screen_height + 50
            elif self.direction == "down":
                self.y += self.speed
                if self.y > screen_height:
                    self.y = -50

    def draw(self):
       
        car_body = pygame.Rect(self.x, self.y, 50, 30)  
        pygame.draw.rect(screen, BLUE, car_body)  
        pygame.draw.circle(screen, BLACK, (self.x + 10, self.y + 30), 5)  
        pygame.draw.circle(screen, BLACK, (self.x + 40, self.y + 30), 5)  


vehicles = []


def add_vehicles():
    lanes = {
        'signal_1': (0, 200, 'right'),
        'signal_2': (screen_width, 300, 'left'),
        'signal_3': (400, screen_height, 'up'),
        'signal_4': (500, 0, 'down')
    }
    
    for signal, (x, y, direction) in lanes.items():
        for _ in range(vehicle_counts[signal]):
            vehicles.append(Vehicle(x, y, signal, direction, signal))

def update_vehicle_counts():
    for signal in vehicle_counts.keys():
        vehicle_counts[signal] = get_vehicle_count(signal)
    add_vehicles()

def open_signal(signal):
    global previous_traffic_lights
    previous_traffic_lights = traffic_lights.copy()
    for s in traffic_lights.keys():
        traffic_lights[s] = 'RED'
    traffic_lights[signal] = 'GREEN'
    recent_green_signals.append(signal)
    print(f"Signal {signal} is GREEN")

def change_signal_to_yellow(signal):
    global previous_traffic_lights
    previous_traffic_lights = traffic_lights.copy()
    traffic_lights[signal] = 'YELLOW'
    print(f"Signal {signal} is YELLOW")

def remove_vehicles(signal):
    global vehicles
    vehicles = [vehicle for vehicle in vehicles if vehicle.signal != signal or traffic_lights[signal] != 'YELLOW']

def display_status():
    while True:
        screen.fill(WHITE)

       
        pygame.draw.rect(screen, GRAY, (0, 190, screen_width, 60))
        pygame.draw.rect(screen, GRAY, (0, 290, screen_width, 60))
        pygame.draw.rect(screen, GRAY, (390, 0, 60, screen_height))
        pygame.draw.rect(screen, GRAY, (490, 0, 60, screen_height))

       
        pole_positions = [(50, 200), (750, 300), (400, 50), (500, 500)]
        signal_names = ['Signal 1', 'Signal 2', 'Signal 3', 'Signal 4']
        
        for idx, pos in enumerate(pole_positions):
            pygame.draw.rect(screen, BLACK, (pos[0], pos[1] - 50, 10, 60))  

            
            current_signal = f'signal_{idx + 1}'
            if traffic_lights[current_signal] == 'RED':
                pygame.draw.circle(screen, RED, (pos[0] + 5, pos[1] - 45), 5)
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 25), 5)  
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 5), 5)  
            elif traffic_lights[current_signal] == 'YELLOW':
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 45), 5) 
                pygame.draw.circle(screen, YELLOW, (pos[0] + 5, pos[1] - 25), 5)
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 5), 5)  
            elif traffic_lights[current_signal] == 'GREEN':
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 45), 5) 
                pygame.draw.circle(screen, GRAY, (pos[0] + 5, pos[1] - 25), 5)  
                pygame.draw.circle(screen, GREEN, (pos[0] + 5, pos[1] - 5), 5)

            
            text_name = font.render(signal_names[idx], True, BLACK)
            screen.blit(text_name, (pos[0] - 10, pos[1] + 20)) 

       
        for idx, (signal, count) in enumerate(vehicle_counts.items()):
            
            text = font.render(f"{signal} Count: {count}", True, BLACK)
            screen.blit(text, (50, 20 + idx * 40))  
            
           
            current_color = traffic_lights[signal]
            text_color = font.render(f"{current_color}", True, BLACK)
            screen.blit(text_color, (280, 20 + idx * 40))  

        
        for vehicle in vehicles:
            vehicle.draw()  
            vehicle.move()

        pygame.display.flip()
        time.sleep(0.1)  

def manage_traffic():
    while True:
        update_vehicle_counts()
        eligible_signals = [signal for signal in vehicle_counts if signal not in recent_green_signals]
        sorted_signals = sorted(eligible_signals, key=lambda s: vehicle_counts[s], reverse=True)
        
        for signal in sorted_signals:
            open_signal(signal)
            time.sleep(15)  
            change_signal_to_yellow(signal)
            remove_vehicles(signal)  
            time.sleep(5) 
            break  

def start_traffic_management():
    traffic_management_thread = threading.Thread(target=manage_traffic)
    traffic_management_thread.start()

def start_display_status():
    status_thread = threading.Thread(target=display_status)
    status_thread.start()

if __name__ == "__main__":
    start_traffic_management()
    start_display_status()

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Signal signal_2 is GREEN


Exception in thread Thread-6 (display_status):
Traceback (most recent call last):
  File "c:\Users\haris\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\haris\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\haris\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\haris\AppData\Local\Temp\ipykernel_12364\4169883897.py", line 136, in display_status
pygame.error: display Surface quit


Signal signal_2 is YELLOW
Signal signal_1 is GREEN
Signal signal_1 is YELLOW
Signal signal_3 is GREEN
Signal signal_3 is YELLOW
Signal signal_4 is GREEN
Signal signal_4 is YELLOW
Signal signal_2 is GREEN
Signal signal_2 is YELLOW
Signal signal_1 is GREEN
Signal signal_1 is YELLOW
Signal signal_3 is GREEN
Signal signal_3 is YELLOW
Signal signal_4 is GREEN
Signal signal_4 is YELLOW
Signal signal_2 is GREEN
Signal signal_2 is YELLOW
Signal signal_1 is GREEN
Signal signal_1 is YELLOW
Signal signal_3 is GREEN
Signal signal_3 is YELLOW
Signal signal_4 is GREEN
Signal signal_4 is YELLOW
Signal signal_2 is GREEN
Signal signal_2 is YELLOW
Signal signal_1 is GREEN
Signal signal_1 is YELLOW
Signal signal_3 is GREEN
Signal signal_3 is YELLOW
Signal signal_4 is GREEN
Signal signal_4 is YELLOW
Signal signal_2 is GREEN
Signal signal_2 is YELLOW
Signal signal_1 is GREEN
Signal signal_1 is YELLOW
Signal signal_3 is GREEN
Signal signal_3 is YELLOW
Signal signal_4 is GREEN
Signal signal_4 is YELLOW
Signa